In [54]:
import os
#import numpy as np
import pandas
from scipy.stats import ttest_ind
#import re
#import urllib
#from bs4 import BeautifulSoup as bs
#from datetime import datetime as dt
#from dateutil.relativedelta import relativedelta as reldelta
#import calendar

debug = 2
print('ok')

ok


In [55]:
os.chdir('/home/idies/workspace/Storage/raddick/raddick_folksonomy/data/secondpass')
music_df = pandas.read_csv('outmusic_secondpass.csv', index_col=[0], encoding='utf-8')
music_df.index.name = 'rownumber'
music_df = music_df.sort_index()
music_df

,filename,source,filetype,band_abbrev,abbrev_type,band,date_format,theyear,themonth,theday,date_right_after_band,mics_etc,score
rownumber,,,,,,,,,,,,,
0,01 Badlands.flac,DimeaDozen,flac,NaN,blank,blank,nodate,NaN,NaN,NaN,True,NaN,0
1,Track_01.flac,DimeaDozen,flac,track,notband,notband,nodate,NaN,NaN,NaN,False,NaN,0
2,Track_01.flac,DimeaDozen,flac,track,notband,notband,nodate,NaN,NaN,NaN,False,NaN,0
3,"d2t01-Heart of Gold, Los Olivos, 2003-09-28.flac",DimeaDozen,flac,d,notband,notband,yyyy-mm-dd,2003.0,9.0,28.0,False,NaN,1
4,01-Immigrant Song.flac,DimeaDozen,flac,NaN,blank,blank,nodate,NaN,NaN,NaN,True,NaN,0
5,06.flac,DimeaDozen,flac,NaN,blank,blank,nodate,NaN,NaN,NaN,True,NaN,0
6,09 a forest.flac,DimeaDozen,flac,NaN,blank,blank,nodate,NaN,NaN,NaN,True,NaN,0
7,01. commentary.flac,DimeaDozen,flac,NaN,blank,blank,nodate,NaN,NaN,NaN,True,NaN,0
8,segment 03.flac,DimeaDozen,flac,segment,jeremy,Segment,nodate,NaN,NaN,NaN,True,NaN,1


# How scores were assigned

In [56]:
#music_df.groupby('date_format').size()
# Full 4 points for: 
#    * band abbrev is canonical, additional, or assigned by Jeremy (correct but not in etree's list)
#    * band is not "unknown"
#    * date directly follows band with no space between
#    * date is correctly formatted as yyyy-mm-dd

'''
music_df.loc[:, 'score'] = -9999

music_df.loc[
    (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'additional', 'jeremy'])) 
    & (music_df['date_format'] == 'yyyy-mm-dd')
#    & (music_df['band'] != 'unknown')
    & (music_df['date_right_after_band'] == True)
    , 'score'] = 4


# Score goes down to 3 if...
#   date is incorrectly formatted (not yyyy-mm-dd) but contains year/month/date in right place...
music_df.loc[
    (music_df['score'] == -9999)
    & (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'additional', 'jeremy'])) 
#    & (music_df['band'] != 'unknown')
    & (music_df['date_format'].apply(lambda x: x not in ['yyyy-mm-dd', 'nodate']))
    & (
        (music_df['theyear'].notnull())
        & (music_df['themonth'].notnull())
        & (music_df['theday'].notnull())
    )
    & (music_df['date_right_after_band'] == True)
, 'score'] = 3

#, OR
# date does not directly follow band name
music_df.loc[
    (music_df['score'] == -9999)
    & (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'additional', 'jeremy'])) 
#    & (music_df['band'] != 'unknown')
    & (music_df['date_format'] == 'yyyy-mm-dd')
    & (music_df['date_right_after_band'] == False)
, 'score'] = 3

#, OR
#   date correct but used an variant band abbreviation
music_df.loc[
    (music_df['score'] == -9999)
    & (music_df['abbrev_type'] == 'variant')
#    & (music_df['band'] != 'unknown')
    & (music_df['date_format'] == 'yyyy-mm-dd')
    & (music_df['date_right_after_band'] == True)
, 'score'] = 3


# Score goes down to 2 if...
#   some date parts are missing
music_df.loc[
    (music_df['score'] == -9999)
    & (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'additional', 'jeremy'])) 
#    & (music_df['band'] != 'unknown')
    & (music_df['date_format'].apply(lambda x: x not in ['yyyy-mm-dd', 'nodate']))
    & (
        (music_df['theyear'].isnull())
        | (music_df['themonth'].isnull())
        | (music_df['theday'].isnull())
    )
    & (music_df['date_right_after_band'] == True)
, 'score'] = 2

#, OR
# date would have scored 3 but it doesn't directly follow band
music_df.loc[
    (music_df['score'] == -9999)
    & (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'additional', 'jeremy'])) 
#    & (music_df['band'] != 'unknown')
    & (music_df['date_format'].apply(lambda x: x not in ['yyyy-mm-dd', 'nodate']))
    & (
        (music_df['theyear'].notnull())
        & (music_df['themonth'].notnull())
        & (music_df['theday'].notnull())
    )
    & (music_df['date_right_after_band'] == False)
, 'score'] = 2

#, OR
#   date correct (ignoring positioning) but used an incorrect band abbreviation
music_df.loc[
    (music_df['score'] == -9999)
    & (music_df['abbrev_type'] == 'incorrect')
#    & (music_df['band'] != 'unknown')
    & (music_df['date_format'] == 'yyyy-mm-dd')
#    & (music_df['date_right_after_band'] == True)
, 'score'] = 2


# Score = 0 if there is no band AND no date
music_df.loc[
    (music_df['score'] == -9999)
    & (music_df['abbrev_type'].apply(lambda x: x in ['blank', 'notband']))
    & (music_df['date_format'] == 'nodate'), 'score'] = 0

# OTHERWISE, score = 1
music_df.loc[music_df['score'] == -9999, 'score'] = 1
'''

"\nmusic_df.loc[:, 'score'] = -9999\n\nmusic_df.loc[\n    (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'additional', 'jeremy'])) \n    & (music_df['date_format'] == 'yyyy-mm-dd')\n#    & (music_df['band'] != 'unknown')\n    & (music_df['date_right_after_band'] == True)\n    , 'score'] = 4\n\n\n# Score goes down to 3 if...\n#   date is incorrectly formatted (not yyyy-mm-dd) but contains year/month/date in right place...\nmusic_df.loc[\n    (music_df['score'] == -9999)\n    & (music_df['abbrev_type'].apply(lambda x: x in ['canonical', 'additional', 'jeremy'])) \n#    & (music_df['band'] != 'unknown')\n    & (music_df['date_format'].apply(lambda x: x not in ['yyyy-mm-dd', 'nodate']))\n    & (\n        (music_df['theyear'].notnull())\n        & (music_df['themonth'].notnull())\n        & (music_df['theday'].notnull())\n    )\n    & (music_df['date_right_after_band'] == True)\n, 'score'] = 3\n\n#, OR\n# date does not directly follow band name\nmusic_df.loc[\n    (music_df['sc

In [57]:
for i in range(4,-1,-1):
    print('One sample file that scored {0:.0f}:'.format(i))
    print(music_df['filename'][music_df['score'] == i].sample(1).values[0])
#    print(music_df[['filename','band_abbrev','abbrev_type','band','date_format','theyear','themonth','theday','date_right_after_band']][music_df['score'] == i].sample(1))
    print('\n')

One sample file that scored 4:
soulemonde2018-01-12set2t03.flac


One sample file that scored 3:
VHF_20170827_13_Let_In_A_Little_Light.flac


One sample file that scored 2:
Phish 2017-08-06 MSG NYC 1ST T-01.flac


One sample file that scored 1:
Tabache 101.flac


One sample file that scored 0:
02.flac




In [49]:
music_df.groupby('score').size()

score
0    9969
1    1816
2     380
3    2422
4    6882
dtype: int64

In [58]:
band_list_df = pandas.DataFrame(data=music_df['band'].drop_duplicates(), columns=['band'])
band_list_df = band_list_df.set_index('band')

band_list_df = band_list_df.assign(count = music_df.groupby('band').size())
band_list_df = band_list_df.assign(count_zero = music_df[music_df['score'] == 0].groupby('band').size())
band_list_df = band_list_df.assign(count_one = music_df[music_df['score'] == 1].groupby('band').size())
band_list_df = band_list_df.assign(count_two = music_df[music_df['score'] == 2].groupby('band').size())
band_list_df = band_list_df.assign(count_three = music_df[music_df['score'] == 3].groupby('band').size())
band_list_df = band_list_df.assign(count_four = music_df[music_df['score'] == 4].groupby('band').size())
band_list_df = band_list_df.assign(mean = music_df.groupby('band')['score'].mean())
band_list_df = band_list_df.assign(median = music_df.groupby('band')['score'].median())
band_list_df = band_list_df.assign(stdev = music_df.groupby('band')['score'].std())
band_list_df = band_list_df.fillna(0)

band_list_df = band_list_df.sort_values(by='count', ascending=False)
band_list_df

,count,count_zero,count_one,count_two,count_three,count_four,mean,median,stdev
band,,,,,,,,,
blank,8497.0,7722.0,775.0,0.0,0.0,0.0,0.091209,0.0,0.287923
notband,2357.0,2246.0,111.0,0.0,0.0,0.0,0.047094,0.0,0.211884
Grateful Dead,2222.0,0.0,210.0,20.0,874.0,1118.0,3.305131,4.0,0.892997
unknown,775.0,0.0,134.0,25.0,116.0,500.0,3.267097,4.0,1.139898
Bob Dylan,439.0,0.0,16.0,4.0,100.0,319.0,3.644647,4.0,0.683406
theSoulutions,372.0,0.0,36.0,18.0,79.0,239.0,3.400538,4.0,0.959166
Jerry Garcia,208.0,0.0,28.0,0.0,180.0,0.0,2.730769,3.0,0.684271
Phish,200.0,0.0,1.0,1.0,52.0,146.0,3.715000,4.0,0.494975
Jerry Garcia Band,179.0,0.0,1.0,2.0,39.0,137.0,3.743017,4.0,0.498192


In [59]:
band_list_df = band_list_df.assign(jamband='no')

jambandlist = ['Grateful Dead', 'The Other Ones', 'The Dead']
jambandlist += ['Phil Lesh & Friends', 'Ratdog', 'Jerry Garcia Band']
jambandlist += ['Jerry Garcia', 'Bruce Hornsby', 'Phish', 'Trey Anastasio Band']
jambandlist += ['Oysterhead']
jambandlist += ['Georgia Avenue & Ratdog', 'Dead & Company']
jambandlist += ['Trey Anastasio (Band)']

jambandmaybelist = ['The Big Wu', 'Blues Traveler', 'Built to Spill']
jambandmaybelist += ['Dark Star Orchestra', 'Dave Matthews Band', 'Dave Matthews']
jambandmaybelist += ['Dave Matthews & Tim Reynolds', 'Drive-By Truckers']
jambandmaybelist += ['Gov\'t Mule', 'Guster', 'Hot Tuna']
jambandmaybelist += ['Les Claypool\'s Fearless Flying Frog Brigade']
jambandmaybelist += ['Col. Claypool\'s Bucket of Bernie Brains']
jambandmaybelist += ['Medeski Martin & Wood', 'moe', 'String Cheese Incident']
jambandmaybelist += ['Umphrey\'s McGee', 'Ween', 'Widespread Panic']
jambandmaybelist += ['Yonder Mountain String Band']
jambandmaybelist += ['Dave Matthews (solo)', 'Tim Reynolds']

band_list_df.loc[band_list_df.index.isin(jambandlist), 'jamband'] = 'yes'
band_list_df.loc[band_list_df.index.isin(jambandmaybelist), 'jamband'] = 'maybe'

music_df = music_df.join(band_list_df['jamband'], how='left', on='band')
print('k')
#band_list_df

k


In [60]:
small_sample_cutoff_value = 5

stats_df = band_list_df[band_list_df['count'] >= small_sample_cutoff_value]

misc_bands = band_list_df[band_list_df['count'] < small_sample_cutoff_value].index.values.tolist()

misc_bands = band_list_df[band_list_df['count'] < small_sample_cutoff_value].index.values.tolist()
misc_count = music_df['score'][music_df['band'].isin(misc_bands)].count()
misc_count_zero = music_df['score'][(music_df['band'].isin(misc_bands)) & (music_df['score'] == 0)].count()
misc_count_one = music_df['score'][(music_df['band'].isin(misc_bands)) & (music_df['score'] == 1)].count()
misc_count_two = music_df['score'][(music_df['band'].isin(misc_bands)) & (music_df['score'] == 2)].count()
misc_count_three = music_df['score'][(music_df['band'].isin(misc_bands)) & (music_df['score'] == 3)].count()
misc_count_four = music_df['score'][(music_df['band'].isin(misc_bands)) & (music_df['score'] == 4)].count()
misc_median = music_df['score'][music_df['band'].isin(misc_bands)].median()
misc_mean = music_df['score'][music_df['band'].isin(misc_bands)].mean()
misc_stdev = music_df['score'][music_df['band'].isin(misc_bands)].std()
allotherbands_df = pandas.DataFrame(data = [[misc_count, misc_count_zero, misc_count_one, misc_count_two, misc_count_three, misc_count_four, misc_median, misc_mean, misc_stdev, 'no']], columns=stats_df.columns, index=['All other bands'])

stats_df = stats_df.append(allotherbands_df)
stats_df


,count,count_zero,count_one,count_two,count_three,count_four,mean,median,stdev,jamband
blank,8497.0,7722.0,775.0,0.0,0.0,0.0,0.091209,0.000000,0.287923,no
notband,2357.0,2246.0,111.0,0.0,0.0,0.0,0.047094,0.000000,0.211884,no
Grateful Dead,2222.0,0.0,210.0,20.0,874.0,1118.0,3.305131,4.000000,0.892997,yes
unknown,775.0,0.0,134.0,25.0,116.0,500.0,3.267097,4.000000,1.139898,no
Bob Dylan,439.0,0.0,16.0,4.0,100.0,319.0,3.644647,4.000000,0.683406,no
theSoulutions,372.0,0.0,36.0,18.0,79.0,239.0,3.400538,4.000000,0.959166,no
Jerry Garcia,208.0,0.0,28.0,0.0,180.0,0.0,2.730769,3.000000,0.684271,yes
Phish,200.0,0.0,1.0,1.0,52.0,146.0,3.715000,4.000000,0.494975,yes
Jerry Garcia Band,179.0,0.0,1.0,2.0,39.0,137.0,3.743017,4.000000,0.498192,yes
Widespread Panic,158.0,0.0,2.0,0.0,7.0,149.0,3.917722,4.000000,0.390414,maybe


In [75]:
group1 = music_df['score'][music_df['jamband'] == 'yes']
group2 = music_df['score'][music_df['jamband'] == 'maybe']

jamband_stats = pandas.DataFrame(data=[['yes'], ['maybe'], ['no']], columns=['jamband'])
jamband_stats = jamband_stats.set_index('jamband')
jamband_stats = jamband_stats.assign(bandcount = band_list_df[band_list_df.index.map(lambda x: x not in ['unknown', 'blank', 'notband'])].groupby('jamband').size())
jamband_stats = jamband_stats.assign(filecount = music_df[music_df['band'].apply(lambda x: x not in ['unknown', 'blank', 'notband'])].groupby('jamband').size())
jamband_stats = jamband_stats.assign(mean_score = music_df[music_df['band'].apply(lambda x: x not in ['unknown', 'blank', 'notband'] )].groupby('jamband')['score'].mean())
jamband_stats = jamband_stats.assign(stdev_score = music_df[music_df['band'].apply(lambda x: x not in ['unknown', 'blank', 'notband'] )].groupby('jamband')['score'].std())

#print(music_df.groupby('jamband').size())

t, p = ttest_ind(group1, group2)
print(p)

jamband_stats

7.63020391201e-12


,bandcount,filecount,mean_score,stdev_score
jamband,,,,
yes,11,2952,3.321138,0.868040
maybe,14,297,3.680135,0.754744
no,2447,6591,3.497952,0.910379
